## BASIC USE OF THE PACKAGE

QuLTRA is an open-source Python library for the analysis of superconducting circuits that include both lumped elements (capacitors, inductors, Josephson junctions, resistors) and distributed components (such as coplanar waveguides and CPW couplers).

The library provides tools to compute:

1) Eigenfrequencies of the circuit modes,

2) Dissipation rates, including coupling to external lines,

3) The cross-Kerr matrix, describing the nonlinear interactions between modes.

QuLTRA is designed to support fast, layout-independent modeling in the early design stages, before full electromagnetic simulations.

To create the circuit, we need to instantiate an object of the QCircuit class.
We must provide:

1) a list of components that define the circuit topology,

2) the minimum and maximum frequencies expressed in GHz over which the circuit will be analyzed.

For lumped components (C, L, J, R), we need to provide:

1) the positive and negative nodes the component connects to,

2) the value of the component (in SI units).

It is also possible to simulate an array of Josephson junction by simply instantiating a J component and passing the number of junctions in the array as the fourth argument.

For a CPW component, we need to provide:

1) the negative and positive nodes it connects to,

2) its length (in meters),

3) and optionally its characteristic impedance (in Ohms).

By default, the impedance is set to 50 Ω if not specified.

**Note**: Node 0 is considered the ground node

For instance, in the case of a qubit coupled to a $\lambda/4$ resonator, we can write:


In [1]:
import numpy as np
import qultra as qu


Cj=90e-15 #qubit capacitance
Lj=8e-9 #qubit junction
Cg=5e-15 #coupling capacitance
l=4.5e-3 #cpw length

f_min=3 #minimum frequency [GHz]
f_max=12 #maximum frequency [GHz]

net=[qu.C(0,1,Cj),qu.J(0,1,Lj,1),qu.C(1,2,Cg),qu.CPW(0,2,l)]

circuit=qu.QCircuit(net,f_min,f_max)


## MODE FREQUENCIES
The mode frequencies together with the dissipation rates are extracted by calculating the zeros of the admittance matrix determinant defined as $\frac{2\pi \kappa}{2}+j2\pi f$.
It is possible to retrieve the mode frequencies (in GHz) as a list—using the mode_frequencies() method—for further processing or numerical calculations,
or simply use the show_modes() method to print them in a nicely formatted table.

The show_modes() method also plots the dissipation rates $\kappa$ for each mode.
Since the circuit contains no dissipative elements, the κ values will be zero, and no quality factor (Q) is defined.

In [ ]:
frequencies = circuit.mode_frequencies()
print("Mode Frequencies (GHz):", frequencies)

circuit.show_modes()

Mode Frequencies (GHz): [5.765693573044772, 6.529970034428903]
+------+------------+----------+
| Mode | Freq [GHz] | k [MHz]  |
+------+------------+----------+
|  1   |  5.77e+00  | 0.00e+00 |
|  2   |  6.53e+00  | 0.00e+00 |
+------+------------+----------+


## CROSS-KERR MATRIX
Since the circuit contains a Josephson junction component, we can also compute the cross-Kerr matrix (expressed in MHz), which describes the nonlinear interactions between the modes.

As with the mode frequencies, the cross-Kerr matrix can either be retrieved as a matrix using the run_epr() method, for further analysis, or visualized directly using the show_chi() method, which plots the matrix in a clear and readable format.

In [3]:
cross_kerr_matrix=circuit.run_epr()
print(cross_kerr_matrix)
circuit.show_chi()

[[1.98809413e+02 5.07443676e+00]
 [5.07443676e+00 3.23801424e-02]]
Chi matrix [MHz]:
+------+----------+----------+
| Mode |    1     |    2     |
+------+----------+----------+
|  1   | 1.99e+02 | 5.07e+00 |
|  2   | 5.07e+00 | 3.24e-02 |
+------+----------+----------+


If we're only interested in visualizing all the key results—including mode frequencies and the cross-Kerr matrix—we can simply call the show_all() method.

This will automatically display:

1) the table of mode frequencies,

2) the dissipation rates (κ),

3) and the cross-Kerr matrix (χ) in a compact and readable format.

In [13]:
circuit.show_all()

+------+------------+----------+
| Mode | Freq [GHz] | k [MHz]  |
+------+------------+----------+
|  1   |  5.77e+00  | 0.00e+00 |
|  2   |  6.53e+00  | 0.00e+00 |
+------+------------+----------+
Chi matrix [MHz]:
+------+----------+----------+
| Mode |    1     |    2     |
+------+----------+----------+
|  1   | 1.99e+02 | 5.07e+00 |
|  2   | 5.07e+00 | 3.24e-02 |
+------+----------+----------+


## DISSIPATION RATE
Typically, quantum circuits are coupled to feedlines terminated with 50 Ω resistors.
This coupling can be either:

1) Capacitive, by using a lumped capacitor that connects the resonator to the feedline

2) Primarily inductive, by using CPW couplers.

QuLTRA allows modeling both types of coupling.
For capacitive coupling, simply add a C component between the resonator node and a 50 Ω resistor (representing the feedline termination).
For CPW coupling, you can instantiate a CPW_coupler component.
This requires:

1) a list of the 4 nodes the coupler is connected to,

2) a list of gap widths (in micrometers),

3) a list of CPW widths involved in the coupling (in micrometers),

4) the length of the coupler (in meters).

Two of the coupler’s terminals are then connected to 50 Ω resistors to model the feedline.

The second case is shown below:



In [2]:
l_coupler=0.7e-3 #coupler length
coupler_nodes=[3,0,4,5] #coupler nodes
gap=[10,10,10,10] #coupler gaps
width=[15,10,15] #coupler widths

net1=[qu.C(0,1,Cj),qu.J(0,1,Lj),qu.C(1,2,Cg),qu.CPW(3,2,l-l_coupler),qu.CPW_coupler(coupler_nodes,gap,width,l_coupler),qu.R(0,4,50),qu.R(0,5,50)]

circuit_with_loss=qu.QCircuit(net1,f_min,f_max)

circuit_with_loss.show_all()

+------+------------+-----------+-----------+
| Mode | Freq [GHz] |  k [MHz]  |     Q     |
+------+------------+-----------+-----------+
|  1   |  5.77e+00  | -1.37e-02 | -4.22e+05 |
|  2   |  6.49e+00  | -1.55e+00 | -4.18e+03 |
+------+------------+-----------+-----------+
Chi matrix [MHz]:
+------+----------+----------+
| Mode |    1     |    2     |
+------+----------+----------+
|  1   | 1.98e+02 | 5.51e+00 |
|  2   | 5.51e+00 | 3.83e-02 |
+------+----------+----------+


As shown in the results, in this case the presence of dissipative components leads to a non-zero coupling rate (κ) and therefore to a finite quality factor (Q).

**Note**: When using a CPW_coupler, the length of the CPW section representing the λ/4 resonator must be adjusted by subtracting the length of the coupler. This ensures the total electrical length remains accurate.